<a href="https://colab.research.google.com/github/takashinnosuke/study_Keras/blob/main/DisneyFaceApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%tensorflow_version 1.x
!git clone https://github.com/justinpinkney/stylegan2

Cloning into 'stylegan2'...
remote: Enumerating objects: 269, done.
remote: Total 269 (delta 0), reused 0 (delta 0), pack-reused 269
Receiving objects: 100% (269/269), 2.32 MiB | 4.03 MiB/s, done.
Resolving deltas: 100% (141/141), done.


In [18]:
%cd stylegan2
!nvcc test_nvcc.cu -o test_nvcc -run

# フォルダーの作成
!mkdir aligned
!mkdir generated

/content/stylegan2/stylegan2/stylegan2
CPU says hello.
cudaErrorNoKernelImageForDevice: no kernel image is available for execution on the device


In [19]:
# ffhqモデル(Baseモデル)とblendedモデルのダウンロード
import pretrained_networks

# use my copy of the blended model to save Doron's download bandwidth
# get the original here https://mega.nz/folder/OtllzJwa#C947mCCdEfMCRTWnDcs4qw
#blended_url = "https://drive.google.com/uc?id=1H73TfV5gQ9ot7slSed_l-lim9X7pMRiU"
blended_url = "https://drive.google.com/uc?id=1BRqqHWk_4BjNHLXTrpHkoxmZhGw3CU59" 
ffhq_url = "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"

_, _, Gs_blended = pretrained_networks.load_networks(blended_url)
_, _, Gs = pretrained_networks.load_networks(ffhq_url)

In [20]:
# 画像表示
import matplotlib.pyplot as plt
from PIL import Image
import glob
import numpy as np

def display_pic(folder):
    fig = plt.figure(figsize=(30, 40))
    files = glob.glob(folder)
    files.sort()
    images=[]
    for i in range(len(files)):
        img = Image.open(files[i])    
        images = np.asarray(img)
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images)
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()  

# 潜在変数(latents)から生成した画像を表示
import PIL.Image
def display(latents):
    
    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
  
    fig = plt.figure(figsize=(30, 40))   
    for i in range(len(latents)):
        vec = latents[i].reshape(1,18,512)
        images =  Gs_blended.components.synthesis.run(vec, randomize_noise=False, **synthesis_kwargs)  
        images = images.transpose((0,2,3,1))     
        PIL.Image.fromarray(images[0], 'RGB')   
        ax = fig.add_subplot(10, 10, i+1, xticks=[], yticks=[])
        image_plt = np.array(images[0])
        ax.imshow(image_plt)
        ax.set_xlabel(str(i), fontsize=20)               
    plt.show()
    plt.close()

# 潜在変数(latents)の順番を指定して、トランジションのGIFを作成する
import os

def generate_gif(latents, idx):

    synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)

    image_gif = []
    os.makedirs('my/gif', exist_ok=True)
    for j in range(len(idx)-1):
        for i in range(20):
            latent = latents[idx[j]]+(latents[idx[j+1]]-latents[idx[j]])*i/19
            latent = latent.reshape(1, 18, 512)
            images =  Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs) 
            images = images.transpose((0,2,3,1)) 
            image_one = PIL.Image.fromarray(images[0], 'RGB')
            image_gif.append(image_one.resize((256,256))) 

    image_gif[0].save('./my/gif/anime.gif', save_all=True, append_images=image_gif[1:],
                      duration=100, loop=0) 

In [21]:
# 顔画像の切り取り
!python align_images.py raw aligned
display_pic('./aligned/*.png')

Traceback (most recent call last):
  File "align_images.py", line 33, in <module>
    for img_name in [x for x in os.listdir(RAW_IMAGES_DIR) if x[0] not in '._']:
FileNotFoundError: [Errno 2] No such file or directory: 'raw'


<Figure size 2160x2880 with 0 Axes>

In [22]:
# 顔画像の潜在変数を求める 
!python project_images.py --num-steps 500 aligned generated
display_pic('./generated/*.png')

Loading networks from "http://d36zk2xti64re0.cloudfront.net/stylegan2/networks/stylegan2-ffhq-config-f.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Preprocessing... Compiling... Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Preprocessing... Compiling... Loading... Done.


<Figure size 2160x2880 with 0 Axes>

In [23]:
# 潜在変数wからアニメ画像を生成する
import numpy as np
from PIL import Image
import dnnlib
import dnnlib.tflib as tflib
from pathlib import Path

latent_dir = Path("generated")
latents = sorted(latent_dir.glob("*.npy"))

for i, latent_file in enumerate(latents):
  latent = np.load(latent_file)
  latent = np.expand_dims(latent,axis=0)
  synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=False), minibatch_size=8)
  images = Gs_blended.components.synthesis.run(latent, randomize_noise=False, **synthesis_kwargs)
  Image.fromarray(images.transpose((0,2,3,1))[0], 'RGB').save(latent_file.parent / (f"{latent_file.stem}-toon.jpg"))
  
  if i == 0:
    latents = latent
  else:
    latents = np.concatenate([latents, latent])  

# 潜在変数(latents)から生成した画像を表示する
display(latents)

<Figure size 2160x2880 with 0 Axes>

In [24]:
# 潜在変数(latents)の順番を指定して、トランジションGIFを作成する
from IPython.display import Image
generate_gif(latents,[0,1,2,0])
Image('./my/gif/anime.gif', format='png')

IndexError: ignored